In [1]:
from keras.models import Sequential , Model
from keras.layers import Dense , Conv2D , MaxPool2D , Flatten
from keras.layers import Input

from keras.utils import to_categorical

from keras.optimizers import Adam , SGD

from glob import glob

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy


%matplotlib inline

In [3]:
BATCH_SIZE = 32

PATH = '../dataset/CIFAR10/train/'

IMAGES_NAME = glob(PATH+'*')

WIDTH = 32
HEIGHT = 32
CHANNEL = 3

IMAGE_SHAPE = (WIDTH , HEIGHT , CHANNEL)

CLASS_NUM = 10 #CIFAR10

In [4]:

LABEL2INDEX = {'frog':0,'truck':1,'deer':2,'automobile':3,'bird':4,'horse':5,'ship':6,'cat':7,'dog':8,'airplane':9}

INDEX2LABEL = {value:key for key,value in LABEL2INDEX.items()}

labels = pd.read_csv('../dataset/CIFAR10/trainLabels.csv')


In [5]:
def load_image():
    images = []
    
    for i in IMAGES_NAME:
        image = scipy.misc.imread(i , mode='RGB').astype(np.float)
        
        images.append(image)
    
    images = np.array(images)/127.5 - 1
    
    return images

In [6]:
def load_label():
    labels = pd.read_csv('../dataset/CIFAR10/trainLabels.csv')

    return to_categorical(labels.label.map(LABEL2INDEX).values , num_classes=10)

In [12]:
images = load_image()

In [28]:
labels = load_label()

In [29]:
#=============

In [7]:
from keras.applications import VGG19

In [8]:
vgg = VGG19()

In [9]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [10]:
vgg.input_shape

(None, 224, 224, 3)

In [11]:
vgg.output_shape

(None, 1000)

In [72]:
#============

In [14]:
model = Sequential()
model.add(Conv2D(filters=64 , input_shape=IMAGE_SHAPE , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=64 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=128 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=128 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=256 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=256 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=256 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=256 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(Conv2D(filters=512 , kernel_size=3 , strides=1 , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2,2) , strides=(2,2)))

model.add(Flatten())

model.add(Dense(4096 , activation='relu'))
model.add(Dense(4096 , activation='relu'))
model.add(Dense(CLASS_NUM , activation='softmax'))

In [20]:
for i in range(0 , 22):
    model.layers[i].set_weights(vgg.layers[i+1].get_weights())
    model.layers[i].trainable = False

In [122]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 8, 8, 256)         295168    
__________

In [129]:
model.compile(optimizer=SGD(lr=0.01 , momentum=0.9) , loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
model.fit(x=images , y=labels , batch_size=256 , epochs=10 , validation_split=0.1)